# Ajuste del Pronóstico de Lluvia del WRF 1.5

### Nota:
Recuerde ajustar los links de las carpetas, directorios y url's que se encuentran a lo largo del código.

## Cargar Librerias
* Se cargan las librerías esenciales para la manipulación de datos, visualización, cálculos numéricos, manejo de fechas, interacción con la web y operaciones del sistema operativo.

In [375]:
import glob # Para buscar archivos en un directorio.
import pandas as pd # Para manipulación de datos.
import numpy as np # Para cálculos numéricos.
from datetime import datetime, date, timedelta # Para fechas y horas.
import requests # Para realizar solicitudes HTTP.
import urllib.request # Para trabajar con URLs.
import io # Para manejar entradas/salidas.
import os # Para interactuar con el sistema operativo.

## Configuración de Almacenamiento y Fechas para Datos de Lluvia
* Se proporciona las de rutas de carpetas ( `carpeta_modelo`, `carpeta_observado`) que se utilizarán para almacenar los datos del modelo y los datos observados de lluvia, respectivamente. Además, se establecen fechas de inicio (`fecha_inicio`) y fin (`fecha_fin`) para cargar archivos de lluvia en un rango específico de días.

* El diccionario `meses_espanol` asocia números de meses con sus nombres en español. Además, se definen enlaces base (`enlace_base_mod` y `enlace_base_obs`) que señalan a los servidores donde se encuentran los datos de lluvia pronosticada y observada.

In [376]:
# Ruta de la carpeta donde se almacenará los datos procesados
carpeta = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Ajuste_Datos/Lluvia/Datos"

# Definición de la ruta de la carpeta donde se almacenarán los datos.
carpeta_modelo = f"{carpeta}/Modelo"
carpeta_observado = f"{carpeta}/Observado"

#Asignar rango de fechas para cargar archivos de lluvia
fecha_inicio = datetime.now().date() - timedelta(days=4)
fecha_fin = datetime.now().date() - timedelta(days=1)

# Diccionario que asocia números de meses con sus nombres en español
meses_espanol = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo", 6: "junio", 7: "julio",
    8: "agosto", 9: "setiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Enlace base al servidor donde se encuentran los datos de lluvia pronosticada.
enlace_base_mod = "https://wrf1-5.imn.ac.cr/modelo/backup/TESIS/"

# Enlace base al servidor donde se encuentran los datos de lluvia observada.
enlace_base_obs = "http://intra-files.imn.ac.cr/Intranet_graficos/datos5minutos/registro024"

In [377]:
fecha_inicio

datetime.date(2023, 12, 3)

In [378]:
fecha_fin

datetime.date(2023, 12, 6)

## Verificar que los archivos de Lluvia (Observado y Modelo) de los 3 días anteriores existan

### Función de Carga y Procesamiento de Datos de Lluvia

La función `cargar_datos` gestiona la carga y procesamiento de archivos relacionados con la lluvia. En primer lugar, utiliza la información de la fecha proporcionada, desglosándola en componentes como año, mes y día, así como el nombre del mes en español. Luego, construye los enlaces completos para los archivos de lluvia observada y pronosticada, utilizando estos componentes y las direcciones base proporcionadas.

La función realiza verificaciones para asegurarse de que los archivos de lluvia observada y pronosticada estén disponibles mediante solicitudes HTTP a los respectivos enlaces. Si ambos archivos están presentes, la función decodifica y lee el archivo CSV de lluvia pronosticada, verificando la presencia de valores negativos o cadenas en los datos. Si se encuentran tales valores problemáticos, se emite un mensaje y el archivo no se carga. En caso contrario, se almacenan los datos observados, los datos pronosticados y la fecha correspondiente en listas específicas.

En situaciones donde uno o ambos archivos no están disponibles, la función emite un mensaje indicando archivos faltantes y agrega la fecha a una lista de datos faltantes. Finalmente, la función devuelve las listas actualizadas de datos pronosticados, observados y las fechas de los archivos procesados. 

In [379]:
# Función para cargar datos y procesar archivos
def cargar_datos(fecha, fecha_anterior, COLUMN_NAMES, enlace_base_obs, enlace_base_mod, meses_espanol):
    
    # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
    año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]

    # Crear el URL completo del archivo de lluvia observada utilizando los componentes de la fecha
    url_observado = f"{enlace_base_obs}/{año}/{mes_text}/{dia}_{mes}_{año}_acum024.txt"

    # Formar el enlace completo para el archivo CSV con la fecha actual y siguiente
    url_modelo = f"{enlace_base_mod}{fecha_anterior.strftime('%Y%m%d')}/{fecha.strftime('%Y%m%d')}.lluvia.csv"

    # Verificar si el archivo de lluvia observada existe antes de descargarlo
    response_obs = requests.get(url_observado)
    response_mod = requests.get(url_modelo)

    if response_obs.status_code == 200 and response_mod.status_code == 200:
        # Decodificar y leer los datos del archivo CSV de lluvia pronosticada
        mod_data = response_mod.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(mod_data), header=None, names=COLUMN_NAMES)

        # Verificar la presencia de valores negativos o cadenas en el archivo de lluvia pronosticada
        negative_values = (df[COLUMN_NAMES[1:]] < 0).any(axis=0)
        str_values = df[COLUMN_NAMES[1:]].applymap(lambda x: isinstance(x, str)).any(axis=0)

        # Si hay valores problemáticos, mostrar un mensaje y no cargar el archivo
        if (negative_values.any() or str_values.any()):
            print(f"Valores de lluvia pronosticada para {fecha} son negativos o del tipo str. No se carga el archivo.")
            missing_data.append(fecha.strftime("%Y%m%d"))
        else:
            # Almacenar datos observados, pronosticados y la fecha en listas correspondientes
            datos_observados[fecha] = response_obs.text
            datos_pronostico.append(df)
            file_date.append(fecha.strftime('%Y%m%d'))
    else:
        # Si falta algún archivo, mostrar un mensaje y agregar la fecha a la lista de datos faltantes
        print(f"Hay archivos faltantes para la fecha {fecha}")
        missing_data.append(fecha.strftime("%Y%m%d"))
    
    return datos_pronostico, datos_observados, file_date

### Gestión de Datos de Lluvia: Descarga, Procesamiento y Manejo de Fechas

Se inicia con la definición de un diccionario `datos_observados` para almacenar el contenido de los archivos de lluvia observada, una lista `datos_pronostico` para almacenar datos de pronóstico, otra lista `file_date` para conservar las fechas de los archivos descargados, y una lista `missing_data` para registrar las fechas de archivos faltantes.

Se especifica un conjunto de nombres de columnas `COLUMN_NAMES` para el DataFrame que se creará a partir de los datos de pronóstico. La variable `fecha_anterior` se inicializa con la fecha de inicio, y a continuación, se inicia un bucle que itera sobre el rango de fechas entre `fecha_inicio` y `fecha_fin`.

Dentro del bucle, se calcula la fecha del archivo a descargar sumando un día a la fecha anterior. Se construyen las URL completas para los archivos de lluvia observada y pronosticada utilizando los componentes de la fecha. Luego, se realizan solicitudes HTTP para verificar la disponibilidad de ambos archivos.

Si algún archivo falta, se muestra un mensaje indicando la ausencia y se agrega la fecha a `missing_data`. También, en este caso, se verifica si es necesario cargar datos de lluvia observada de cuatro días atrás.

En caso de que ambos archivos estén disponibles, se decodifica y lee el archivo CSV de lluvia pronosticada. Se verifica la presencia de valores negativos o cadenas en los datos y, si se detectan, se emite un mensaje y se añade la fecha a la lista `missing_data`. Además, si hay menos de dos archivos faltantes, se intenta cargar datos de lluvia observada de cuatro días atrás utilizando la función `cargar_datos`.

Finalmente, la fecha anterior se actualiza para la próxima iteración del bucle.

In [380]:
datos_observados = {} # Diccionario para almacenar el contenido de los archivos de lluvia observada

datos_pronostico = [] # Lista para almacenar los datos de pronóstico

file_date = [] # Lista para almacenar las fechas de los archivos descargados

missing_data = []# Lista para almacenar las fechas de archivos faltantes

COLUMN_NAMES = ["Estacion", "1", "2", "3", "4"] # Nombres de columnas para el DataFrame

fecha_anterior = fecha_inicio # Inicializar la fecha anterior con la fecha de inicio

# Iterar sobre el rango de fechas
while fecha_anterior < fecha_fin:

    fecha = fecha_anterior + timedelta(days=1) # Calcular la fecha del archivo a descargar sumando un día a la fecha anterior

    # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
    año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]

    # Crear el URL completo del archivo de lluvia observada utilizando los componentes de la fecha
    url_observado = f"{enlace_base_obs}/{año}/{mes_text}/{dia}_{mes}_{año}_acum024.txt"

    # Formar el enlace completo para el archivo CSV con la fecha actual y siguiente
    url_modelo = f"{enlace_base_mod}{fecha_anterior.strftime('%Y%m%d')}/{fecha.strftime('%Y%m%d')}.lluvia.csv"

    # Verificar si el archivo de lluvia observada y del modelo existe antes de descargarlo
    response_obs = requests.get(url_observado)
    response_mod = requests.get(url_modelo)

    # Verificar si ambos archivos están disponibles
    if response_obs.status_code == 200 and response_mod.status_code == 200:
        # Decodificar y leer los datos del archivo CSV de lluvia pronosticada
        mod_data = response_mod.content.decode('utf-8')
        df = pd.read_csv(io.StringIO(mod_data), header=None, names=COLUMN_NAMES)

        # Verificar la presencia de valores negativos o cadenas en el archivo de lluvia pronosticada
        negative_values = (df[COLUMN_NAMES[1:]] < 0).any(axis=0)
        str_values = df[COLUMN_NAMES[1:]].applymap(lambda x: isinstance(x, str)).any(axis=0)

        # Si hay valores problemáticos, mostrar un mensaje y no cargar el archivo
        if (negative_values.any() or str_values.any()):
            print(f"Valores de lluvia pronosticada para la fecha {fecha} son negativos o del tipo str. No se carga el archivo.")
            missing_data.append(fecha.strftime("%Y%m%d")) # Agregar la fecha del archivo faltante

            # Verificar si es necesario cargar datos de lluvia observada de cuatro días atrás
            if len(missing_data) < 2:
                fecha_sust = fecha_inicio  # Obtener la fecha siguiente
                fecha_sust_anterior = fecha_inicio - timedelta(days=1)
                datos_pronostico, datos_observados, file_date = cargar_datos(fecha_sust, fecha_sust_anterior, COLUMN_NAMES, enlace_base_obs, enlace_base_mod, meses_espanol)
            else:
                print(f"Hay más de dos archivos faltantes")
        else:
            # Almacenar datos observados, pronosticados y la fecha en listas correspondientes
            datos_observados[fecha] = response_obs.text
            datos_pronostico.append(df)
            file_date.append(fecha.strftime('%Y%m%d'))
    else:
        # Si falta algún archivo, mostrar un mensaje y agregar la fecha a la lista de datos faltantes
        print(f"Hay archivos faltantes para la fecha {fecha}")
        missing_data.append(fecha.strftime("%Y%m%d"))

        # Verificar si es necesario cargar datos de lluvia observada de cuatro días atrás
        if len(missing_data) < 2:
            fecha_sust = fecha_inicio  # Obtener la fecha siguiente
            fecha_sust_anterior = fecha_inicio - timedelta(days=1)
            datos_pronostico, datos_observados, file_date = cargar_datos(fecha_sust, fecha_sust_anterior, COLUMN_NAMES, enlace_base_obs, enlace_base_mod, meses_espanol)
        else:
            print(f"Hay más de dos archivos faltantes")

    # Actualizar la fecha anterior para la próxima iteración del ciclo
    fecha_anterior = fecha

## Dar formato a datos de pronóstico de Lluvia del WRF 1.5

Se realiza una serie de operaciones en el conjunto de datos de pronóstico `datos_pronostico`. En cada iteración, se procesa un DataFrame (`df`) que contiene información de lluvia pronosticada para una fecha específica. 

En primer lugar, se sustituyen los valores de tipo cadena (`str`) por NaN en las columnas "1", "2", "3" y "4". Luego, se convierten a valores numéricos las columnas mencionadas, y se remplazan los valores menores a cero por NaN, asegurando así la coherencia del conjunto de datos.

A continuación, se calcula la suma de las columnas "1", "2", "3" y "4" para obtener la cantidad total de lluvia pronosticada, y se almacena el resultado en una nueva columna llamada 'Lluvia'. Se seleccionan las columnas 'Estacion' y 'Lluvia' para formar un nuevo DataFrame denominado `data`.

Posteriormente, el DataFrame `data` se guarda en dos archivos CSV y TXT con nombres generados basados en la fecha de pronóstico. Estos archivos se almacenan en la carpeta del modelo, específicamente en una subcarpeta llamada 'WRF_Crudo'.

Finalmente, se añade el DataFrame `data` a una lista llamada `data_modelo`, que se utiliza para almacenar los DataFrames filtrados correspondientes al modelo.

In [381]:
data_modelo = []
for date, df in zip(file_date,datos_pronostico):

    # Sustituir los valores str por NaN en las columnas
    df[["1", "2", "3", "4"]] = df[["1", "2", "3", "4"]].apply(pd.to_numeric, errors='coerce')

    # Convertir valores menores a cero en las columnas "1", "2", "3" y "4" en NaN
    df[["1", "2", "3", "4"]] = df[["1", "2", "3", "4"]].applymap(lambda x: np.nan if x < 0 else x)

    # Calcular la suma de las columnas 1, 2, 3 y 4 y almacenar el resultado en la columna 'Lluvia'
    df['Lluvia'] = df[["1", "2", "3", "4"]].sum(axis=1, skipna=True)
    
    data = df[['Estacion', 'Lluvia']]

    # Guardar el DataFrame en un archivo CSV con el nombre generado
    data.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{date}.lluvia.csv"), index=False)
    data.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{date}.lluvia.txt"), index=False)
    
    # Crear una lista de DataFrames filtrados para el modelo
    data_modelo.append(data)

## Procesamiento y Almacenamiento de Datos Observados
En primer lugar, se crea una lista vacía llamada `data_observada`, esta lista se utilizará para almacenar los DataFrames resultantes de los datos observados. Luego, utiliza un bucle para iterar a través de las parejas de `file_date` (fechas en formato texto) y el contenido de archivos previamente guardados en `datos_observados`.

Dentro del bucle, el contenido del archivo se divide en líneas utilizando `"\n"` como separador. Se omite la primera línea, que suele ser la cabecera, mediante el uso de `.strip().split("\n")[1:]`. 

A continuación, se inicia una lista llamada `estaciones` para almacenar los datos extraídos de las líneas del archivo. El bucle procesa cada línea y divide sus datos por comas utilizando el método `.split(",")`. Se verifica si la línea no está vacía y si contiene exactamente 5 campos de datos, lo que indica que tiene la información necesaria. Si esto se cumple, se extraen la identificación de la estación y la cantidad de lluvia registrada durante 24 horas. Luego, estos datos se agregan a la lista `estaciones`

Una vez se han procesado todas las líneas del archivo, la lista `estaciones` se convierte en un DataFrame de Pandas llamado `df_estaciones` con las columnas "Estacion" y "Valor", que representan la estación y la cantidad de lluvia registrada, respectivamente.

Finalmente, el DataFrame `df_estaciones` se agrega a la lista `data_observada`, que contiene los archivos de los 3 días anteriores en el formato adecuado.

In [382]:
# Crear una lista para almacenar los DataFrames de datos de lluvia observada luego de ajustar el formato
data_observada = []

# Iterar sobre las fechas y DataFrames de lluvia observada
for date, archivo in zip(file_date, datos_observados.values()):
    # Dividir el contenido del DataFrame en líneas usando "\n" como separador
    lineas = archivo.strip().split("\n")[1:]  # Omitir la primera línea que es la cabecera

    # Inicializar una lista para almacenar los datos de las estaciones del DataFrame
    estaciones = []

    # Procesar cada línea para extraer los datos de las estaciones
    for linea in lineas:
        datos_estacion = linea.split(",")  # Dividir la linea por comas

        # Asegurarnos de que la línea no esté vacía y tenga todos los campos necesarios
        if datos_estacion and len(datos_estacion) == 5:
            # Asignar a cada variable la columna adecuada
            estacion = datos_estacion[0].split("-")[0] #Dividir por guiones "-" y asignar el valor 0
            lluvia024 = datos_estacion[3] #Asignar la columna 3

            # Solo usar las lineas donde lluvia024 es diferente a 'NA'
            if lluvia024 != "NA": 
                estacion = int(estacion) # Convertir el valor a tipo int
                lluvia024 = float(lluvia024) # Convertir el valor a tipo float64

                # Agregar los datos de la estación y lluvia a la lista estaciones
                estaciones.append((estacion, lluvia024))

    # Convertir la lista de estaciones en un DataFrame de Pandas para este archivo
    df_estaciones = pd.DataFrame(estaciones, columns=["Estacion", "Lluvia"])

    # Guardar el DataFrame en un archivo CSV con el nombre generado
    df_estaciones.to_csv(os.path.join(carpeta_observado, 'Crudo', f"{date}.lluvia.csv"), index=False)
    df_estaciones.to_csv(os.path.join(carpeta_observado, 'Crudo', f"{date}.lluvia.txt"), index=False)

    # Agregar el DataFrame con el nuevo formato a la lista de DataFrames
    data_observada.append(df_estaciones)## Procesamiento y Almacenamiento de Datos Observados

## Crear Función para Cargar las Tablas de la Climatología Mensual

La función `cargar_clim` está diseñada para facilitar la carga de tablas de climatología mensual de la lluvia desde un archivo Excel. El usuario debe proporcionar la ruta del archivo Excel y una lista de nombres de hojas que desea cargar. La función utiliza la biblioteca Pandas para leer cada hoja especificada, convierte los nombres de las columnas a cadenas de texto y organiza los resultados en un diccionario. Este diccionario tiene como claves los nombres de las hojas y como valores los DataFrames correspondientes, permitiendo un acceso sencillo y organizado a los datos climatológicos cargados. 

In [383]:
def cargar_clim(ruta_excel, hojas_excel):
    """
    Cargar las tablas de la climatología mensual de la lluvia observada
    
    Parameters:
        ruta_excel (str): Ruta del archivo Excel que contiene las tablas.
        hojas_excel (list): Lista de nombres de hojas de Excel a cargar.

    Returns:
        dict: Un diccionario que contiene las tablas cargadas. Las claves son los nombres de las hojas y los valores son DataFrames de Pandas.
    """
    # Crear un diccionario para almacenar las tablas de la climatología mensual (lluvia observada) cargadas desde Excel
    clim = {}
    
    # Iterar sobre la lista de nombres de hojas de Excel a cargar
    for hoja in hojas_excel:
        # Cargar la tabla desde la hoja de Excel especificada
        tabla = pd.read_excel(ruta_excel, sheet_name=hoja)
        
        # Convertir los nombres de las columnas a cadenas de texto
        tabla.columns = tabla.columns.astype(str)
        
        # Agregar la tabla al diccionario utilizando el nombre de la hoja como clave
        clim[hoja] = tabla

    return clim

### Carga de Tablas Climatología Mensual Observada

En el código proporcionado, se define la variable `ruta_observado` como la ruta completa de un archivo Excel que contiene la climatología mensual de la lluvia observada. Además, se especifica una lista de nombres de hojas relevantes dentro de ese archivo, que incluyen 'min', 'max' y 'mean'. Luego, se utiliza la función `cargar_clim` con los parámetros `ruta_observado` y `hojas_observado` para cargar las tablas correspondientes desde el archivo Excel.

In [384]:
# Ruta del archivo Excel que contiene la climatología mensual de lluvia observada.
ruta_observado = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Observado/Lluvia/Climatologia_Mensual_Observada_Lluvia.xlsx'
hojas_observado = ['min', 'max', 'mean']

#Cargar las tablas de la climatología mensual de la lluvia observada
clim_observado = cargar_clim(ruta_observado, hojas_observado)

### Carga de Tablas Climatología Mensual del Modelo

En el código proporcionado, se define la variable `ruta_modelo` como la ruta completa de un archivo Excel que contiene la climatología mensual de la lluvia pronosticada. Además, se especifica una lista de nombres de hojas relevantes dentro de ese archivo, que incluyen 'min', 'max' y 'mean'. Luego, se utiliza la función `cargar_clim` con los parámetros `ruta_modelo` y `hojas_modelo` para cargar las tablas correspondientes desde el archivo Excel.

In [385]:
# Ruta del archivo Excel que contiene la climatología mensual de lluvia observada.
ruta_modelo = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Modelo/Lluvia/Climatologia_Mensual_Modelo_Lluvia.xlsx'
hojas_modelo = ['mean']

#Cargar las tablas de la climatología mensual de la lluvia observada
clim_modelo = cargar_clim(ruta_modelo, hojas_modelo)

## Control de Calidad con Climatología Mensual Observada

### Datos de Pronóstico
Se inicia verificando que haya datos de lluvia pronosticada (`data_modelo`) antes de continuar con el proceso. Esto evita errores y asegura que el código se ejecute solo cuando haya información para procesar. Luego, el bucle `for` itera a través de las fechas (`file_date`) y sus respectivos DataFrames de pronóstico (`data_modelo`).

Se crea o carga la tabla de control de calidad (`tabla_cc_lluvia`) es fundamental. La verificación de la existencia del archivo CSV asociado (`ruta_archivo_cc`) asegura que la tabla se inicie correctamente y, en caso de existir, se carga la versión previa para su actualización.

Se procede a eliminar los datos antiguos relacionados con la fecha actual para actualizar los valores. Al verificar si la fecha ya está presente en la tabla, se evita la duplicación de información.

Se crea una nueva fila en `tabla_cc_lluvia` para la fecha actual y se ajustan los valores de lluvia pronosticada en función de los límites establecidos por la climatología mensual de lluvia observada. Se itera sobre las columnas de `tabla_cc_lluvia`, cada columna representa una estación meteorológica.

Para cada estación y el mes correspondiente, se obtienen los valores mínimo (valor_min) y máximo (valor_max) de lluvia observada de la climatología mensual. Se buscan las filas en el DataFrame pronosticado (file) que coinciden con la estación actual.

Se procede con el control de calidad del valor pronosticado (valor_modelo) con los límites establecidos por la climatología observada. Si el valor pronosticado es mayor que el máximo, se ajusta y se registra en la nueva fila (nueva_fila), y se actualiza el valor en file. Si el valor es negativo, también se ajusta. Si el valor está dentro de los límites, se marca con un guion ("-"). Si no hay datos pronosticados para la estación en cuestión, se asigna "x" en la nueva fila, indicando que no hay datos disponibles para esa estación.

Se guarda la tabla de control de calidad ajustada en archivos CSV y TXT. La utilización de nombres de archivo basados en el año de la fecha actual facilita la organización para que cada año se cree una nueva tabla.

La parte final del código aborda la situación en la que `data_modelo` está vacío. En este caso, se emite un mensaje informativo que señala la ausencia de datos y sugiere que no se pudo realizar el proceso correctamente.

In [386]:
# Verificar si data_modelo es None
if data_modelo is not None:
    for date, file in zip(file_date, data_modelo):

        fecha = datetime.strptime(date, "%Y%m%d")
        
        # Ruta completa del archivo que contendrá la tabla de control de calidad
        ruta_archivo_cc = os.path.join(carpeta, f"tabla_cc_lluvia_{fecha.year}.csv")

        # Si el archivo no existe, se crea la tabla vacía y se descargan y procesan los datos
        if not os.path.exists(ruta_archivo_cc):
            # Continuar con el procesamiento
            tabla_cc_lluvia = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_modelo[0]['Estacion']])
        else:
            # Si el archivo existe, cargar la tabla y procesar el día siguiente
            tabla_cc_lluvia = pd.read_csv(ruta_archivo_cc)

        # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
        fecha_siguiente_int = np.int64(date)

        # Verificar si la fecha ya existe en la tabla
        if fecha_siguiente_int in tabla_cc_lluvia["FECHA"].values:
            # Si la fecha ya existe, obtener el índice de la fila existente
            idx_fecha_existente = tabla_cc_lluvia.index[tabla_cc_lluvia["FECHA"] == fecha_siguiente_int][0]

            # Eliminar la fila existente a partir del índice
            tabla_cc_lluvia = tabla_cc_lluvia.drop(idx_fecha_existente)

        # Crear una nueva fila con la fecha siguiente
        nueva_fila = {col: "" for col in tabla_cc_lluvia.columns}
        nueva_fila["FECHA"] = date

        for estacion in tabla_cc_lluvia.columns[1:]:
            if estacion in clim_observado['min'].columns and estacion in clim_observado['max'].columns:
                # Obtener los valores de climatología mensual de lluvia observada para la estación y el mes correspondiente
                valor_min = clim_observado['min'].loc[clim_observado['min']["mes"] == fecha.month, estacion].values[0]
                valor_max = clim_observado['max'].loc[clim_observado['max']["mes"] == fecha.month, estacion].values[0]

                # Verificar si existen filas que coincidan con la estación en file
                filas_estacion = file.loc[file["Estacion"].astype(str).str.strip() == estacion.strip()]

                # Verificar que la fila para la 'estación' en lluvia pronosticada no esté vacía
                if not filas_estacion.empty:
                    # Obtener el valor de lluvia pronosticada para la estación y el mes correspondiente
                    valor_modelo = filas_estacion["Lluvia"].iloc[0]

                    # Control calidad de la lluvia pronosticada con los límites de la climatología mensual de lluvia observada
                    if valor_modelo >= valor_max:
                        nueva_fila[estacion] = f"{valor_max}({valor_modelo}>Max)"
                        file.loc[filas_estacion.index, "Lluvia"] = valor_max
                    elif valor_modelo < 0:
                        nueva_fila[estacion] = f"{valor_min}({valor_modelo}<0)"
                        file.loc[filas_estacion.index, "Lluvia"] = valor_min
                    else:
                        nueva_fila[estacion] = "-"  # Si el valor está dentro de los límites, asignar "-"
                else:
                    # Si no hay valor para la estación en file, asignar "x"
                    nueva_fila[estacion] = "x"
                    print(f'La estación {estacion} no existe en los datos de pronostico de lluvia del WRF1.5')
            else:
                nueva_fila[estacion] = "x"
                print(f'La estación {estacion} no existe en las columnas de la Climatología de Lluvia Mensual Observada')

        # Convertir los diccionarios en Pandas DataFrames
        nueva_fila = pd.DataFrame.from_dict(nueva_fila, orient='index').T

        # Concatenar los DataFrames
        tabla_cc_lluvia = pd.concat([tabla_cc_lluvia, nueva_fila], ignore_index=True)

        # Guardar la tabla ajustada en un archivo CSV y TXT
        tabla_cc_lluvia.to_csv(f"{carpeta}/tabla_cc_lluvia_{fecha.year}.csv", index=False)
        tabla_cc_lluvia.to_csv(os.path.join(carpeta, f"tabla_cc_lluvia_{fecha.year}.txt"), index=False)
else:
        print("No se pudieron obtener los datos correctamente ya que el archivo que contiene los datos de pronostico (data_modelo) está vacio.")

### Datos Observados
Primero se comprueba si la variable `data_observada` no es `None`. Si tiene datos, procede con el procesamiento. En caso contrario, imprime un mensaje indicando que no se pudieron obtener los datos correctamente.

Para cada fecha en `file_date` y su respectivo archivo en `data_observada_cc`: Convierte la fecha a un objeto `datetime`, genera la ruta completa del archivo que contendrá la tabla de control de calidad (`ruta_archivo_cc`), verifica si el archivo ya existe. Si no existe, crea una tabla vacía. Si existe, carga la tabla existente. Convierte la fecha a un número entero para facilitar la comparación con fechas en la tabla.

Se verifica si la fecha ya existe en la tabla de Control de Calidad. Si existe, elimina la fila existente. Srea una nueva fila con la fecha y procesa los datos de lluvia pronosticada y observada para cada estación.

Par}a el control de calidad de datos de lluvia observada, se compara los valores de lluvia observada con los límites de la climatología mensual de lluvia observada y se actualiza los valores en el archivo original (`file`), marcando aquellos que no cumplen con los límites como `NaN`.

Guarda la tabla ajustada en un archivo CSV y TXT. Guarda el archivo original con los datos actualizados en una carpeta específica, utilizando el nombre de la fecha como parte del nombre del archivo.

Imprime mensajes de advertencia si una estación no existe en los datos observados de lluvia o en las columnas de la climatología mensual observada.

In [387]:
# Verificar si data_observada es None
if data_observada is not None:
    data_observada_cc = data_observada.copy()
    for date, file in zip(file_date, data_observada_cc):

        fecha = datetime.strptime(date, "%Y%m%d")
        
        # Ruta completa del archivo que contendrá la tabla de control de calidad
        ruta_archivo_cc = os.path.join(carpeta, f"tabla_cc_lluvia_observada_{fecha.year}.csv")

        # Si el archivo no existe, se crea la tabla vacía y se descargan y procesan los datos
        if not os.path.exists(ruta_archivo_cc):
            # Continuar con el procesamiento
            tabla_cc_lluvia_observada = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_observada_cc[0]['Estacion']])
        else:
            # Si el archivo existe, cargar la tabla y procesar el día siguiente
            tabla_cc_lluvia_observada = pd.read_csv(ruta_archivo_cc)

        # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
        date_siguiente_int = np.int64(date)

        # Verificar si la fecha ya existe en la tabla
        if date_siguiente_int in tabla_cc_lluvia_observada["FECHA"].values:
            # Si la fecha ya existe, obtener el índice de la fila existente
            idx_date_existente = tabla_cc_lluvia_observada.index[tabla_cc_lluvia_observada["FECHA"] == date_siguiente_int][0]

            # Eliminar la fila existente a partir del índice
            tabla_cc_lluvia_observada = tabla_cc_lluvia_observada.drop(idx_date_existente)

        # Crear una nueva fila con la fecha siguiente
        nueva_fila = {col: "" for col in tabla_cc_lluvia_observada.columns}
        nueva_fila["FECHA"] = date

        for estacion in tabla_cc_lluvia_observada.columns[1:]:
            if estacion in clim_observado['min'].columns and estacion in clim_observado['max'].columns:
                # Obtener los valores de climatología mensual de lluvia observada para la estación y el mes correspondiente
                valor_min = clim_observado['min'].loc[clim_observado['min']["mes"] == fecha.month, estacion].values[0]
                valor_max = clim_observado['max'].loc[clim_observado['max']["mes"] == fecha.month, estacion].values[0]

                # Verificar si existen filas que coincidan con la estación en file
                filas_estacion = file.loc[file["Estacion"].astype(str).str.strip() == estacion.strip()]

                # Verificar que la fila para la 'estación' en lluvia pronosticada no esté vacía
                if not filas_estacion.empty:
                    # Obtener el valor de lluvia pronosticada para la estación y el mes correspondiente
                    valor_observado = filas_estacion["Lluvia"].iloc[0]

                    # Control calidad de la lluvia pronosticada con los límites de la climatología mensual de lluvia observada
                    if valor_observado >= valor_max:
                        nueva_fila[estacion] = f"{valor_max}({valor_observado}>Max)"
                        file.loc[filas_estacion.index, "Lluvia"] = np.nan
                    elif valor_observado < 0:
                        nueva_fila[estacion] = f"{valor_min}({valor_observado}<0)"
                        file.loc[filas_estacion.index, "Lluvia"] = np.nan
                    else:
                        nueva_fila[estacion] = "-"  # Si el valor está dentro de los límites, asignar "-"
                else:
                    # Si no hay valor para la estación en file, asignar "x"
                    nueva_fila[estacion] = "x"
                    print(f'La estación {estacion} no existe en los datos observados de lluvia')
            else:
                nueva_fila[estacion] = "x"
                print(f'La estación {estacion} no existe en las columnas de la Climatología de Lluvia Mensual Observada')

        # Convertir los diccionarios en Pandas DataFrames
        nueva_fila = pd.DataFrame.from_dict(nueva_fila, orient='index').T

        # Concatenar los DataFrames
        tabla_cc_lluvia_observada = pd.concat([tabla_cc_lluvia_observada, nueva_fila], ignore_index=True)

        # Guardar la tabla ajustada en un archivo CSV y TXT
        tabla_cc_lluvia_observada.to_csv(f"{carpeta}/tabla_cc_lluvia_observada_{fecha.year}.csv", index=False)
        tabla_cc_lluvia_observada.to_csv(os.path.join(carpeta, f"tabla_cc_lluvia_observada_{fecha.year}.txt"), index=False)
        
        # Guardar el DataFrame en un archivo CSV con el nombre generado
        file.to_csv(os.path.join(carpeta_observado, 'Editado', f"{date}.lluvia.csv"), index=False)
        file.to_csv(os.path.join(carpeta_observado, 'Editado', f"{date}.lluvia.txt"), index=False)
else:
        print("No se pudieron obtener los datos correctamente ya que el archivo que contiene los datos de pronostico (data_observada) está vacio.")

La estación 84237 no existe en las columnas de la Climatología de Lluvia Mensual Observada
La estación 84221 no existe en las columnas de la Climatología de Lluvia Mensual Observada
La estación 84233 no existe en las columnas de la Climatología de Lluvia Mensual Observada
La estación 73159 no existe en los datos observados de lluvia
La estación 96003 no existe en los datos observados de lluvia
La estación 90013 no existe en las columnas de la Climatología de Lluvia Mensual Observada
La estación 84293 no existe en las columnas de la Climatología de Lluvia Mensual Observada
La estación 73173 no existe en las columnas de la Climatología de Lluvia Mensual Observada
La estación 69751 no existe en las columnas de la Climatología de Lluvia Mensual Observada
La estación 72193 no existe en los datos observados de lluvia
La estación 69725 no existe en los datos observados de lluvia
La estación 73137 no existe en los datos observados de lluvia
La estación 84169 no existe en las columnas de la Cli

## Pasos previos al Ajuste de Datos con Climatología Mensual

### Crear un MAPEO que permite relacionar las cabeceras de cantón con estaciones especificas

* Se define un diccionario llamado `mapeo`. Cada clave del diccionario representa el nombre de un cantón de Costa Rica, y el valor asociado a esa clave es una lista de estaciones que se asocian por ubicación a ese cantón.

In [388]:
mapeo = {
    'ALAJUELA': ['84185', '84187'],
    'ABANGARES': ['78031'],
    'ACOSTA': ['88055'],
    'AGUIRRE': ['90015'],
    'ALAJUELITA': ['84203'],
    'ALFARO_RUIZ': ['69737'],
    'ALVARADO': ['73141'],
    'ASERRI': ['84215', '84217'],
    'ATENAS': ['84225'],
    'BAGACES': ['76055'],
    'BARVA': ['84243', '84197'],
    'BELEN': ['84199'],
    'BUENOS_AIRES': ['98087'],
    'CANAS': ['76055'],
    'CARRILLO': ['74067'],
    'CARTAGO': ['73123'],
    'CD_QUESADA': ['69743', '69701'],
    'CORREDORES': ['100651'],
    'COTO_BRUS': ['98107', '98109', '98075'],
    'CURRIDABAT': ['84203'],
    'DESAMPARADOS': ['84203'],
    'DOTA': ['88051'],
    'EL_GUARCO': ['73123', '73171'],
    'ESPARZA': ['80013'],
    'ESCAZU': ['84231', '84193'],
    'FLORES': ['84199', '84197'],
    'GARABITO': ['86013'],
    'GOICOECHEA': ['84139'],
    'GOLFITO': ['100643'],
    'GRECIA': ['84295'],
    'GUACIMO': ['73145'],
    'GUAPILES': ['73147'],
    'GUATUSO': ['69747', '69749'],
    'HEREDIA': ['84193', '84243', '84197'],
    'HOJANCHA': ['72185'],
    'JIMENEZ': ['73149'],
    'LA_CRUZ': ['72191'],
    'LA_UNION': ['84181', '73129', '84249'],
    'LEON_CORTEZ': ['88051', '88047'],
    'LIBERIA': ['74063'],
    'LIMON': ['81005'],
    'LOS_CHILES': ['69633', '69713'],
    'MATINA': ['73159', '83007'],
    'MONTES_DE_OCA': ['84139', '84203'],
    'MONTES_DE_ORO': ['78033'],
    'MORA': ['84209', '84283'],
    'MORAVIA': ['84139'],
    'NANDAYURE': ['72189'],
    'NARANJO': ['84239'],
    'NICOYA': ['72165', '72183'],
    'OREAMUNO': ['73123', '73129'],
    'OROTINA': ['82011', '82019', '82017'],
    'OSA': ['100655'],
    'PALMARES': ['84239'],
    'PARAISO': ['73123'],
    'PARRITA': ['88049'],
    'PAVAS': ['84193', '84285'],
    'PEREZ_ZELEDON': ['98097', '94015'],
    'POAS': ['84189', '84187'],
    'POCOCI': ['73147', '73169'],
    'PTO_VIEJO': ['85025', '85023'],
    'PUNTARENAS': ['78027'],
    'PURISCAL': ['84209'],
    'QUEPOS': ['90015'],
    'SAN_CARLOS': ['69743', '69701'],
    'SAN_ISIDRO': ['84243'],
    'SAN_JOSE': ['84141'],
    'SAN_MARCOS': ['88051'],
    'SAN_MATEO': ['82011'],
    'SAN_PABLO': ['84243', '84193'],
    'SAN_RAFAEL': ['84243'],
    'SAN_RAMON': ['84239'],
    'SANTA_ANA': ['84219'],
    'SANTA_BARBARA': ['84197'],
    'SANTA_CRUZ': ['74053'],
    'SANTO_DOMINGO': ['84193'],
    'SARAPIQUI': ['69725'],
    'SIQUIRRES': ['73159'],
    'SIXAOLA': ['87013'],
    'TALAMANCA': ['85021'],
    'TARRAZU': ['88051'],
    'TIBAS': ['84141'],
    'TILARAN': ['76063'],
    'TORTUGUERO': ['71023'],
    'TURRIALBA': ['73151', '73155', '73167'],
    'TURRUBARES': ['84225'],
    'UPALA': ['69679', '69647'],
    'VALVERDE_VEGA': ['84239'],
    'VASQUEZ': ['84207', '84213']
}

### Mapeo Invertido

Se construye un diccionario inverso llamado `mapeo_invertido`. Inicialmente, se tiene un diccionario `mapeo` que asocia cantones con estaciones meteorológicas. 

El código realiza un bucle sobre cada elemento del diccionario `mapeo`, en cada iteración para cada estación asociado a un canton, se verifica si dicho código ya está presente en el diccionario inverso `mapeo_invertido`. Si la estación ya existe, se agrega el cantón actual a la lista existente de cantones asociadas a esa estación. En caso contrario, se crea una nueva entrada en `mapeo_invertido` con la estación como clave y una lista que contiene el cantón actual como único elemento.

El resultado final es un diccionario inverso que asocia cada estación con una lista de cantones de Costa Rica.

In [389]:
mapeo_invertido = {}

for ubicacion, codigos in mapeo.items():
    for codigo in codigos:
        if codigo in mapeo_invertido:
            mapeo_invertido[codigo].append(ubicacion)
        else:
            mapeo_invertido[codigo] = [ubicacion]

### Lista de nombres de las cabeceras de cantón

* Crear una lista llamada `names_cantones` que contiene los nombres de cada cantón acomodados en el mismo orden que las columnas del diccionario `clim_modelo`. Lo anterior permitirá renombrar las columnas de `clim_modelo` luego para que tengan un formato correcto y homogeneo. 

In [390]:
names_cantones = ['ALAJUELA', 'ABANGARES', 'ACOSTA', 'AGUIRRE', 'ALAJUELITA','ALFARO_RUIZ', 'ALVARADO', 'ASERRI', 'ATENAS', 
           'BAGACES', 'BARVA','BELEN', 'BUENOS_AIRES', 'CANAS', 'CARRILLO', 'CARTAGO', 'CD_QUESADA','CORREDORES', 
           'COTO_BRUS', 'CURRIDABAT', 'DESAMPARADOS', 'DOTA','EL_GUARCO', 'ESPARZA', 'ESCAZU', 'FLORES', 'GARABITO', 
           'GOICOECHEA','GOLFITO', 'GRECIA', 'GUACIMO', 'GUAPILES', 'GUATUSO', 'HEREDIA','HOJANCHA', 'JIMENEZ', 'LA_CRUZ', 
           'LA_UNION', 'LEON_CORTEZ', 'LIBERIA','LIMON', 'LOS_CHILES', 'MATINA', 'MONTES_DE_OCA', 'MONTES_DE_ORO','MORA', 
           'MORAVIA', 'NANDAYURE', 'NARANJO', 'NICOYA', 'OREAMUNO','OROTINA', 'OSA', 'PALMARES', 'PARAISO', 'PARRITA', 
           'PAVAS','PEREZ_ZELEDON', 'POAS', 'POCOCI', 'PTO_VIEJO', 'PUNTARENAS','PURISCAL', 'QUEPOS', 'SAN_CARLOS', 
           'SAN_ISIDRO', 'SAN_JOSE','SAN_MARCOS', 'SAN_MATEO', 'SAN_PABLO', 'SAN_RAFAEL', 'SAN_RAMON','SANTA_ANA', 
           'SANTA_BARBARA', 'SANTA_CRUZ', 'SANTO_DOMINGO','SARAPIQUI', 'SIQUIRRES', 'SIXAOLA', 'TALAMANCA', 'TARRAZU', 
           'TIBAS','TILARAN', 'TORTUGUERO', 'TURRIALBA', 'TURRUBARES', 'UPALA', 'VALVERDE_VEGA', 'VASQUEZ']

###  Integración y Promedio de datos por Estación usando el Mapeo Invertido

Inicialmente, extrae la tabla 'mean' y la transforma en un nuevo DataFrame llamado `df`. Posteriormente, elimina la columna 'mes' de este nuevo DataFrame y renombra sus columnas utilizando nombres específicos de cantones.

Se crea un DataFrame vacío llamado `Clim_Modelo_Mean` que servirá para almacenar los resultados finales. 

Se itera sobre las columnas de `mapeo_invertido`, donde cada estación meteorológica se asigna a una lista de cantones. Dependiendo de si la lista de cantones tiene un solo elemento o más, se asigna directamente el valor correspondiente de `df` a la columna de `Clim_Modelo_Mean` o calcula la media de los valores para manejar el caso de múltiples cantones. 

In [391]:
# Obtener la tabla 'mean' de 'clim_modelo' y convertirla en un nuevo DataFrame 'df'
df = clim_modelo['mean']
df = pd.DataFrame(df)

# Eliminar la columna 'mes' del nuevo DataFrame 'df'
df = df.drop('mes', axis=1)

# Renombrar las columnas del DataFrame 'df' con los nombres de los cantones
df.columns = names_cantones

# Crear un nuevo DataFrame 'Clim_Modelo_Mean' para almacenar los resultados finales
Clim_Modelo_Mean = pd.DataFrame()

# Iterar sobre las columnas del diccionario 'mapeo_invertido'
for estacion, cantones in mapeo_invertido.items():
    # Verificar si hay un solo elemento en la lista de cantones
    if len(cantones) == 1:
        Clim_Modelo_Mean[estacion] = df[cantones[0]]  # Asignar directamente el valor correspondiente
    else:
        # Si hay más de un elemento, calcular la media y manejar valores nulos
        Clim_Modelo_Mean[estacion] = df[cantones].mean(axis=1, skipna=True)

### Obtener la diferencia entre la climatología del modelo y la observada para cada cantón.

Se calculan las diferencias entre las medias de dos conjuntos de datos: `Clim_Modelo_Mean` y `Clim_Observ_Mean`. Los resultados de estas restas se almacenan en un nuevo DataFrame llamado `Tabla_Resta_Clim`. Además, se agrega la columna 'mes' al DataFrame resultante.

In [392]:
# Obtener la tabla 'mean' de 'clim_observado' y asignarla a 'Clim_Observ_Mean'
Clim_Observ_Mean = clim_observado['mean']

# Crear un nuevo DataFrame 'Tabla_Resta_Clim' para almacenar los resultados de las restas
Tabla_Resta_Clim = pd.DataFrame()

# Iterar sobre las columnas del DataFrame 'Clim_Modelo_Mean'
for estacion in Clim_Modelo_Mean.columns:
    # Calcular la resta entre las medias de 'Clim_Modelo_Mean' y 'Clim_Observ_Mean' para cada estación
    resta = Clim_Modelo_Mean[estacion] - Clim_Observ_Mean[estacion]
    
    # Asignar directamente la columna de restas al DataFrame 'Tabla_Resta_Clim'
    Tabla_Resta_Clim[estacion] = resta

# Agregar la columna 'mes' al DataFrame 'Tabla_Resta_Clim'
Tabla_Resta_Clim['mes'] = clim_observado['mean']['mes']

## Ajuste de Datos con Climatología Mensual Observada y Modelada

Se inicia verificando si la variable `data_modelo` no es nula. En caso afirmativo, se procede a un bucle que itera sobre cada fecha y su respectivo archivo en `data_modelo`. Para cada iteración, se convierte la fecha de formato string a un objeto `datetime`, y se construye la ruta completa del archivo que contendrá la tabla de ajuste, denominado `tabla_ajuste_lluvia`.

A continuación, se evalúa si dicho archivo ya existe. En caso de no existir, se inicializa una nueva tabla de ajuste, `tabla_ajuste_lluvia`, con columnas correspondientes a las fechas y estaciones presentes en los datos modelados.

Luego, se procede a iterar sobre las estaciones presentes en `tabla_ajuste_lluvia`. Para cada estación, se verifica si esta existe en la tabla `nueva_tabla` (contiene los valores de la resta de "Climatología Mensual del Modelo menos Climatología Mensual Observada") y, en caso afirmativo, se obtiene el valor correspondiente.

Posteriormente, se buscan las filas en el archivo de lluvia pronosticada (`file`) que coincidan con la estación actual. Si se encuentran filas correspondientes, se recupera el valor de lluvia pronosticada para la estación y el mes actual. Este valor se ajusta restando la diferencia entre la climatología modelada y observada, y se actualiza el valor en `file`.

La tabla de ajuste `tabla_ajuste_lluvia` también se actualiza con los valores ajustados. Además, se registran en esta tabla los ajustes realizados para cada estación.

Finalmente, se guardan tanto la tabla de ajuste como los archivos ajustados en formatos CSV y TXT. Este proceso se repite para cada fecha y su respectivo archivo en `data_modelo`. En el caso de que `data_modelo` sea nulo, se emite un mensaje indicando que los datos no se obtuvieron correctamente.

In [393]:
# Verificar si data_modelo es None
if data_modelo is not None:
    for date, file in zip(file_date, data_modelo):

        fecha = datetime.strptime(date, "%Y%m%d")
        
        # Ruta completa del archivo que contendrá la tabla de ajuste
        ruta_archivo_ajuste = os.path.join(carpeta, f"tabla_ajuste_lluvia_{fecha.year}.csv")

        # Si el archivo no existe, se crea la tabla vacía y se descargan y procesan los datos
        if not os.path.exists(ruta_archivo_ajuste):
            # Continuar con el procesamiento
            tabla_ajuste_lluvia = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_modelo[0]['Estacion']])
        else:
            # Si el archivo existe, cargar la tabla y procesar el día siguiente
            tabla_ajuste_lluvia = pd.read_csv(ruta_archivo_ajuste)
        
        # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
        fecha_siguiente_int = np.int64(date)

        # Verificar si la fecha ya existe en la tabla
        if fecha_siguiente_int in tabla_ajuste_lluvia["FECHA"].values:
            # Si la fecha ya existe, obtener el índice de la fila existente
            idx_fecha_existente = tabla_ajuste_lluvia.index[tabla_ajuste_lluvia["FECHA"] == fecha_siguiente_int][0]

            # Eliminar la fila existente a partir del índice
            tabla_ajuste_lluvia = tabla_ajuste_lluvia.drop(idx_fecha_existente)

        # Crear una nueva fila con la fecha siguiente
        nueva_fila = {col: "" for col in tabla_ajuste_lluvia.columns}
        nueva_fila["FECHA"] = date

        # Iterar sobre las columnas de tabla_ajuste_lluvia
        for estacion in tabla_ajuste_lluvia.columns[1:]:
            # Verificar que la estación exista en Tabla_Resta_Clim
            if estacion in Tabla_Resta_Clim.columns:
                # Filtrar según el valor de 'mes' la Tabla_Resta_Clim
                valor_resta = Tabla_Resta_Clim.loc[Tabla_Resta_Clim["mes"] == fecha.month, estacion].values[0]

                # Verificar si existen filas que coincidan con la estación en data_modelo
                filas_estacion = file.loc[file["Estacion"].astype(str).str.strip() == estacion.strip()]

                # Verificar que la fila para la 'estación' en data_modelo (lluvia pronosticada) no esté vacía
                if not filas_estacion.empty:
                    # Obtener el valor de lluvia pronosticada para la estación y el mes correspondiente
                    valor_modelo = filas_estacion["Lluvia"].iloc[0]

                    # Ajustar el valor de lluvia pronosticada restando la diferencia entre la climatología modelada y observada
                    nuevo_valor = valor_modelo - valor_resta
                    if nuevo_valor < 0:
                        # Actualizar el valor en data_modelo con el ajuste realizado
                        file.loc[filas_estacion.index, "Lluvia"] = 0

                        # Añadir el nuevo valor ajustado a la nueva fila de tabla_ajuste_lluvia
                        nueva_fila[estacion] = f"{0}({valor_modelo})"
                    else:
                        # Actualizar el valor en data_modelo con el ajuste realizado
                        file.loc[filas_estacion.index, "Lluvia"] = nuevo_valor

                        # Añadir el nuevo valor ajustado a la nueva fila de tabla_ajuste_lluvia
                        nueva_fila[estacion] = f"{nuevo_valor}({valor_modelo})"

                else:
                    # Si no hay valor para la estación en data_modelo, asignar "x"
                    nueva_fila[estacion] = "x"
            else:
                nueva_fila[estacion] = "-"

        # Convertir la nueva_fila de tabla_ajuste_lluvia en Pandas DataFrames
        nueva_fila = pd.DataFrame.from_dict(nueva_fila, orient='index').T

        # Concatenar la nueva_fila con tabla_ajuste_lluvia
        tabla_ajuste_lluvia = pd.concat([tabla_ajuste_lluvia, nueva_fila], ignore_index=True)

        # Guardar la tabla ajustada en un archivo CSV y TXT
        tabla_ajuste_lluvia.to_csv(os.path.join(carpeta, f"tabla_ajuste_lluvia_{fecha.year}.csv"), index=False)
        tabla_ajuste_lluvia.to_csv(os.path.join(carpeta, f"tabla_ajuste_lluvia_{fecha.year}.txt"), index=False)
        
        # Guardar el DataFrame en un archivo CSV con el nombre generado
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajust_Clim', f"{date}.lluvia.csv"), index=False)
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajust_Clim', f"{date}.lluvia.txt"), index=False)

else:
    print("No se pudieron obtener los datos correctamente.")

## Ajuste de datos con el promedio de la resta de los 3 días anteriores

### Crear o Abrir Tabla de Promedios de Resta de los 3 días anteriores

Se genera una fecha para el dia siguiente (`fecha`), una fecha anterior (hoy) (`fecha_anterior`), y se extrae el año correspondiente (`ano`). Se define el nombre de la tabla que se creará o abrirá para contener el promedio de la diferencia entre pronóstico y observado. Luego, se verifica si la carpeta especificada existe; si no, se crea. Se construye la ruta completa del archivo de la tabla, y si el archivo no existe, se inicializa la tabla vacía con la columna 'Estacion' del primer conjunto de datos en `data_modelo`. En caso de que `data_modelo` sea `None`, se imprime un mensaje de error. Si el archivo ya existe, los datos se cargan desde el archivo CSV.

In [394]:
# Obtener la fecha actual y la fecha anterior
fecha = datetime.now().date() + timedelta(days=1)
fecha_anterior = datetime.now().date()

# Obtener el año actual
ano = fecha.year

# Definir el nombre de la tabla que contendrá el promedio de la diferencia (pronóstico - observado)
nombre_tabla = f"tabla_promedios_lluvia_{ano}"

# Comprobar si la carpeta existe; si no existe, crearla
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

# Construir la ruta completa del archivo que contendrá la tabla
ruta_archivo = os.path.join(carpeta, f"{nombre_tabla}.csv")

# Si el archivo no existe, crear la tabla vacía
if not os.path.exists(ruta_archivo):
    if data_modelo is not None:
        # Inicializar la tabla con la columna 'Estacion' del primer conjunto de datos en 'data_modelo'
        tabla_promedios_lluvia = data_modelo[0][["Estacion"]]
    else:
        # Imprimir un mensaje de error si no se pudieron obtener los datos correctamente
        print("No se pudieron obtener los datos correctamente.")
else:
    # Si el archivo existe, cargar los datos desde el archivo CSV
    tabla_promedios_lluvia = pd.read_csv(ruta_archivo)

### Verificar si hay datos del pronóstico ajustados para los 3 dían anteriores

Se especifica la ruta de dos carpetas. La primera carpeta, "carpeta_wrf_ajust," contiene datos ajustados del modelo WRF, mientras que la segunda carpeta, "carpeta_obs_crudo," almacena datos observados.

La ejecución del código está condicionada a la existencia y validez de la carpeta "carpeta_wrf_ajust." Si la carpeta existe y es un directorio válido, el programa procede a verificar la cantidad de archivos en dicha carpeta. Si hay al menos 8 archivos, se inicia un proceso de carga de datos.

Para cada fecha en la lista de fechas de pronóstico ("file_date"), el código construye las rutas completas de los archivos de pronóstico y observados en las carpetas correspondientes. Si ambos archivos existen, se leen en pandas DataFrames y se agregan a las listas "data_modelo" y "data_observada" respectivamente. También, la fecha se agrega a la lista "dates."

En caso de que un archivo no se encuentre en las carpetas especificadas, se imprime un mensaje indicando la ausencia del archivo para esa fecha.

Si la carpeta "carpeta_wrf_ajust" tiene menos de 8 archivos, se imprime un mensaje indicando la insuficiencia de archivos y se informa que no se cargarán datos ajustados del WRF. Si la carpeta no existe o no es un directorio válido, se emiten mensajes informando de esta situación y también se indica que no se cargarán archivos ajustados del WRF.

Finalmente, si las variables "file_date" o "data_modelo" no contienen datos, se imprime un mensaje indicando que una o ambas tablas están vacías.

In [395]:
# Especificar las rutas de las carpetas que contiene los datos
carpeta_wrf_ajust = f"{carpeta}/Modelo/WRF_Ajustado/"
carpeta_obs_crudo = f"{carpeta}/Observado/Crudo/"

if file_date and data_modelo:
    # Verificar si la carpeta existe y es un directorio válido
    if os.path.exists(carpeta_wrf_ajust) and os.path.isdir(carpeta_wrf_ajust):
        # Obtener la lista de archivos en la carpeta
        archivos_en_carpeta = os.listdir(carpeta_wrf_ajust)

        # Contar la cantidad de archivos en la carpeta
        cantidad_archivos = len(archivos_en_carpeta)

        # Verificar si hay 8 o más archivos en la carpeta
        if cantidad_archivos >= 8:
            print(f"La carpeta tiene 8 o más archivos. Cantidad de archivos: {cantidad_archivos}")
            data_modelo, data_observada, dates = [], [], []

            # Iterar sobre las fechas de pronóstico
            for date in file_date:
                # Crear la ruta completa del archivo de pronóstico en el directorio local
                ruta_wrf_ajust = os.path.join(carpeta_wrf_ajust, f"{date}.lluvia.csv")
                ruta_obs_crudo = os.path.join(carpeta_obs_crudo, f"{date}.lluvia.csv")

                # Verificar si el archivo existe en la carpetas
                if os.path.exists(ruta_wrf_ajust) and os.path.exists(ruta_obs_crudo):
                    # Leer el archivo CSV y agregar los datos al conjunto de datos 'data_modelo'
                    file_wrf = pd.read_csv(ruta_wrf_ajust)
                    data_modelo.append(file_wrf)
                    # Leer el archivo CSV y agregar los datos al conjunto de datos 'data_observada'
                    file_obs = pd.read_csv(ruta_obs_crudo)
                    data_observada.append(file_obs)
                    # Agregar las fechas de datos a 'dates'
                    dates.append(date)
                else:
                    # Imprimir un mensaje si el archivo no se encuentra en la carpeta de datos ajustados del WRF
                    print(f'El archivo para el {date} no se encuentra en la carpeta de datos del WRF ajustados o observados')
        else:
            # Imprimir un mensaje si la carpeta tiene menos de 8 archivos
            print(f"La carpeta tiene menos de 8 archivos. Cantidad de archivos: {cantidad_archivos}")
            print("No se cargan archivos del WRF ya ajustados")
    else:
        # Imprimir un mensaje si la carpeta no existe o no es un directorio válido
        print("La carpeta no existe o no es un directorio válido.")
        print("No se cargan archivos del WRF ya ajustados")
else:
    print("Alguna de las tablas o ambas estan vacias")

La carpeta tiene 8 o más archivos. Cantidad de archivos: 28


### Obtener la resta (pronostico menos observado) de los 3 días anteriores

Se crea una lista llamada `Tablas_diferencias` para almacenar las tablas que contendrán las diferencias calculadas. Luego, se recorren las listas `pro` y `obs` en un bucle para calcular las diferencias entre los DataFrames correspondientes y guardar los resultados en archivos CSV.

Dentro del bucle, se realiza una combinación "left" entre los DataFrames `data_modelo[i]` y `data_observada[i]` basada en la columna "Estacion". Esto combina los datos de ambas fuentes en un solo DataFrame llamado `diferencias`.

Se calculan las diferencias restando los valores de lluvia de los DataFrames `data_observada` de los valores de `data_modelo`, y se agregan estas diferencias como una nueva columna llamada "Resta" en el DataFrame `diferencias`.

Luego, se seleccionan las columnas necesarias ("Estacion" y "Resta") del DataFrame `diferencias` y se guarda este DataFrame en la lista `Tablas_diferencias`.

In [396]:
# Crear una lista para almacenar los resultados (lluvia pronosticada menos lluvia observada)
tablas_diferencias = []

if data_modelo and data_observada:
    # Recorrer las listas de DataFrames (pro y obs) para calcular las diferencias (pro - obs) para cada día
    for i in range(len(data_modelo)):

        # Realizar una combinación "left" entre pro y obs en base a la columna "Estacion"
        diferencias = data_modelo[i].merge(data_observada[i], on="Estacion", how="left", suffixes=("_pro", "_obs"))

        # Calcular las diferencias (pro - obs) y guardarlo en la columna 'Resta'
        diferencias["Resta"] = diferencias["Lluvia_pro"] - diferencias["Lluvia_obs"]

        # Filtrar para seleccionar solo las columnas necesarias
        diferencias = diferencias[["Estacion", "Resta"]]
        
        # Eliminar filas con valores NaN en la columna 'Resta'
        diferencias = diferencias.dropna(subset=["Resta"])

        # Agregar la tabla diferencias a la lista tablas_diferencias
        tablas_diferencias.append(diferencias)
else:
    print("Alguna de las tablas (datos_pronostico y data_observada) o ambas estan vacias")## Pronostico menos Observado

### Obtener el promedio de la resta de los 3 días anteriores

Se empieza por concatenar todos los DataFrames presentes en `Tablas_diferencias` en un solo DataFrame llamado `tabla_concatenada`. Esta operación combina todas las tablas de diferencias en una única tabla, lo que facilita el procesamiento conjunto.

Después, la función calcula el promedio de las diferencias para cada estación en la tabla concatenada. Esta información se almacena en el DataFrame `tabla_promedio`. La columna del promedio se renombra con el valor de `text_hoy`, que representa el día actual.

Se verifica si la columna con el nombre de `text_hoy` ya existe en el DataFrame `tabla_promedios_lluvia`, si esta columna existe, se elimina para evitar duplicados y garantizar la integridad de los datos.

A continuación, se fusionan los DataFrames `tabla_promedios_lluvia` y `tabla_promedio` en base a la columna "Estacion", utilizando un método de fusión "outer". Esto asegura que se incluyan todas las estaciones presentes en ambos DataFrames en la tabla final, manteniendo la coherencia de los datos.

La tabla resultante, que ahora contiene los promedios de las diferencias junto con los datos históricos, se guarda en un archivo CSV en la ubicación especificada por `carpeta`.

In [397]:
from datetime import datetime, date, timedelta  # Importamos clases relacionadas con fechas y horas.

if tablas_diferencias:
    # Concatenar las tres tablas de tablas_diferencias en una sola tabla
    tabla_concatenada = pd.concat(tablas_diferencias)

    # Calcular el promedio agrupando por Estación
    tabla_promedio = tabla_concatenada.groupby("Estacion")["Resta"].mean().reset_index()

    # Obtener la fecha actual en formato texto y renombrar la columna 'Resta' de tabla_promedio
    text = fecha.strftime("%Y%m%d")
    tabla_promedio = tabla_promedio.rename(columns={"Resta": f"{text}"})

    # Verificar si la columna ya existe en tabla_promedios_lluvia
    if f"{text}" in tabla_promedios_lluvia.columns:
        # Si existe la columna existe, eliminarla.
        tabla_promedios_lluvia.drop(columns=[f"{text}"], inplace=True)

    # Fusionar tabla_promedios_lluvia con tabla_promedio
    tabla_promedios_lluvia = tabla_promedios_lluvia.merge(tabla_promedio, on="Estacion", how="outer")

    # Guardar la tabla_promedios_lluvia en un archivo CSV y TXT
    tabla_promedios_lluvia.to_csv(f"{carpeta}/tabla_promedios_lluvia_{ano}.csv", index=False)
    tabla_promedios_lluvia.to_csv(os.path.join(carpeta, f"tabla_promedios_lluvia_{ano}.txt"), index=False)
else:
    print("Tabla diferencias está vacía")

### Cargar pronóstico de mañana

Primero, se construye el enlace completo utilizando una URL base, la fecha actual y la fecha siguiente. Luego, se utiliza la biblioteca `requests` para verificar la existencia del archivo en la URL especificada. Si la respuesta del servidor es exitosa (código 200), se procede a procesar el contenido del archivo.

El código decodifica el contenido CSV y lo convierte en un DataFrame de Pandas. Se realiza una verificación para detectar valores negativos o de tipo cadena en las columnas relevantes del DataFrame. Si se encuentran tales valores, se imprime un mensaje indicando que el archivo no se cargará. En caso contrario, se efectúan diversas operaciones en el DataFrame para manipular y limpiar los datos.

Se sustituyen los valores de tipo cadena por NaN en ciertas columnas, se convierten a NaN los valores numéricos inferiores a cero, y se calcula la suma de las columnas correspondientes, almacenando el resultado en una nueva columna llamada 'Lluvia'. Finalmente, se crea un nuevo DataFrame con las columnas 'Estacion' y 'Lluvia' y se guarda este DataFrame en archivos CSV y TXT en una ubicación específica.

Si la verificación inicial de la existencia del archivo en la URL no es exitosa, se imprime un mensaje indicando que hay archivos faltantes para la fecha actual.

In [398]:
# Formar el enlace completo para el archivo CSV con la fecha actual y siguiente¿
url_modelo_mañana = f"{enlace_base_mod}{fecha_anterior.strftime('%Y%m%d')}/{fecha.strftime('%Y%m%d')}.lluvia.csv"

# Verificar si el archivo de lluvia observada existe antes de descargarlo
response = requests.get(url_modelo_mañana)

if response.status_code == 200:
    csv_data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(csv_data), header=None, names=COLUMN_NAMES)

    negative_values = (df[COLUMN_NAMES[1:]] < 0).any(axis=0)
    str_values = df[COLUMN_NAMES[1:]].applymap(lambda x: isinstance(x, str)).any(axis=0)

    if (negative_values.any() or str_values.any()):
        print(f"Valores de lluvia pronosticada para {fecha} son negativos o del tipo str. No se carga el archivo.")
    else:
        # Sustituir los valores str por NaN en las columnas
        df[["1", "2", "3", "4"]] = df[["1", "2", "3", "4"]].apply(pd.to_numeric, errors='coerce')

        # Convertir valores menores a cero en las columnas "1", "2", "3" y "4" en NaN
        df[["1", "2", "3", "4"]] = df[["1", "2", "3", "4"]].applymap(lambda x: np.nan if x < 0 else x)

        # Calcular la suma de las columnas 1, 2, 3 y 4 y almacenar el resultado en la columna 'Lluvia'
        df['Lluvia'] = df[["1", "2", "3", "4"]].sum(axis=1, skipna=True)

        file = df[['Estacion', 'Lluvia']]

        # Guardar el DataFrame en un archivo CSV con el nombre generado
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{fecha.strftime('%Y%m%d')}.lluvia.csv"), index=False)
        file.to_csv(os.path.join(carpeta_modelo, 'WRF_Crudo', f"{fecha.strftime('%Y%m%d')}.lluvia.txt"), index=False)
else:
    print(f"Hay archivos faltantes para la fecha {fecha.strftime('%Y%m%d')}")

### Restar al pronóstico de mañana el promedio de la resta de los 3 días previos

Se realiza un ajuste en los pronósticos de lluvia para el día siguiente, utilizando información de una tabla de promedios. Mediante un condicional se encarga de verificar la existencia de las variables `tabla_promedio` y `file` en el entorno local. Si ambas variables existen y contienen datos, se procede con el ajuste.

Primero, se fusiona el pronóstico del día siguiente con la tabla de promedios, relacionando las estaciones meteorológicas. Luego, se realiza una operación de resta, restando el promedio de los últimos tres días a la lluvia pronosticada para el día de mañana. Se asegura de que el resultado no sea menor que 0.

Posteriormente, se filtran las columnas necesarias, en este caso, "Estacion" y "Lluvia". El DataFrame resultante se guarda en archivos CSV y TXT en una ubicación específica dentro de la carpeta del modelo ajustado.

También maneja casos donde las tablas o archivos involucrados puedan estar vacíos o no definidos. Se imprimen mensajes de advertencia en tales situaciones.

In [399]:
if "tabla_promedio" in locals() and "file" in locals():
    if tabla_promedio is not None and file is not None:
        if not tabla_promedio.empty and not file.empty:
            # Unir el pronóstico de mañana con la tabla del promedio (pro - obs) de los 3 días previos
            Pronostico = file.merge(tabla_promedio, on="Estacion", how="left")
            
            # Restarle al pronóstico de mañana el promedio de los 3 días previos y asegurarse de que el resultado no sea menor que 0
            Pronostico["Lluvia"] = Pronostico.apply(lambda row: max(row["Lluvia"] - row[text] if not np.isnan(row[text]) else row["Lluvia"], 0), axis=1)

            # Filtrar para seleccionar solo las columnas necesarias
            Pronostico = Pronostico[["Estacion", "Lluvia"]]

            # Guardar el DataFrame del ajuste del pronóstico de mañana en archivos CSV y TXT
            Pronostico.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajustado', f"{fecha.strftime('%Y%m%d')}.lluvia.csv"), index=False)
            Pronostico.to_csv(os.path.join(carpeta_modelo, 'WRF_Ajustado', f"{fecha.strftime('%Y%m%d')}.lluvia.txt"), index=False)
        else:
            print("Alguna de las tablas (tabla_promedio y file) o ambas están vacías")
    else:
        print("Alguna de las tablas (tabla_promedio y file) no está definida")
else:
    print("Alguna de las tablas (tabla_promedio y file) no existe en el entorno actual.")

In [400]:
Pronostico

,Estacion,Lluvia
0,69633,2.634037
1,69647,0.000000
2,69679,3.904494
3,69699,8.422333
4,69701,4.168444
...,...,...
91,100641,13.851400
92,100649,1.395333
93,100651,1.062067
94,100653,0.000000
